🔹 1. Imports & Configurations

In [ ]:
# === Basic Utilities ===
import os, random, shutil, warnings
from pathlib import Path
from collections import defaultdict
from collections import Counter

# === Data Handling ===
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from time import time

# === Visualization ===
import seaborn as sns
import matplotlib.pyplot as plt

# === Image Handling ===
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.simplefilter('error', Image.DecompressionBombWarning)

# === TensorFlow & Keras ===
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers


# === CLIP (for filtering) ===
import clip
import torch
from tqdm import tqdm

from sklearn.cluster import DBSCAN
from collections import Counter, defaultdict

# === Paths ===
metadata_path = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\metadata.csv"
source_dir = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species1"
dest_dir = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_filtered"
dest_dir_clip = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_final"
dest_dir_removed = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_removed"

c:\Users\Elcano\anaconda3\envs\cv310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load metadata
metadata = pd.read_csv(metadata_path)
"""
This cell loads the metadata CSV file and analyzes the distribution of images across families.
It counts and calculates the percentage of images per family to assess dataset balance.
The output helps identify potential class imbalances that may need addressing.
"""
# Calculate image counts per family
family_stats = metadata['family'].value_counts().reset_index()
family_stats.columns = ['Family', 'Image Count']
family_stats['Percentage'] = (family_stats['Image Count'] / family_stats['Image Count'].sum()) * 100

# Display results
print("Dataset Balance by Family:")
print(family_stats.to_string(index=False))

Dataset Balance by Family:
           Family  Image Count  Percentage
  cercopithecidae          300    2.503547
      dactyloidae          300    2.503547
       formicidae          291    2.428440
   plethodontidae          270    2.253192
   carcharhinidae          270    2.253192
    salamandridae          270    2.253192
          bovidae          240    2.002837
      bucerotidae          210    1.752483
      acroporidae          210    1.752483
         anatidae          180    1.502128
      diomedeidae          180    1.502128
         atelidae          150    1.251773
          laridae          150    1.251773
           apidae          150    1.251773
        bufonidae          150    1.251773
       dasyatidae          150    1.251773
      delphinidae          120    1.001419
      agariciidae          120    1.001419
   callitrichidae          120    1.001419
      psittacidae          120    1.001419
     spheniscidae          120    1.001419
        iguanidae          

In [3]:
"""
This cell loads the metadata CSV file and analyzes the distribution of images across families.
It counts and calculates the percentage of images per family to assess dataset balance.
The output helps identify potential class imbalances that may need addressing.
"""
# Calculate image counts per family
family_stats = metadata['phylum'].value_counts().reset_index()
family_stats.columns = ['phylum', 'Image Count']
family_stats['Percentage'] = (family_stats['Image Count'] / family_stats['Image Count'].sum()) * 100

# Display results
print("Dataset Balance by phylum:")
print(family_stats.to_string(index=False))

Dataset Balance by phylum:
       phylum  Image Count  Percentage
     chordata         9952   83.050989
   arthropoda          951    7.936243
     cnidaria          810    6.759576
     mollusca          210    1.752483
echinodermata           60    0.500709


🔹 2. Image Filtering (Low Quality & Corrupted)

In [5]:
valid_formats = ['jpeg', 'jpg', 'png', 'bmp']
min_size_kb = 5
max_pixels = 89_478_485

os.makedirs(dest_dir, exist_ok=True)

total_images = valid_images = removed_small = removed_too_large = removed_invalid = 0

for family_folder in os.listdir(source_dir):
    src_path = os.path.join(source_dir, family_folder)
    dst_path = os.path.join(dest_dir, family_folder)
    os.makedirs(dst_path, exist_ok=True)

    for file in os.listdir(src_path):
        total_images += 1
        file_path = os.path.join(src_path, file)
        try:
            if os.path.getsize(file_path) < min_size_kb * 1024:
                removed_small += 1
                continue
            with Image.open(file_path) as img:
                if img.format.lower() not in valid_formats:
                    removed_invalid += 1
                    continue
                if img.width * img.height > max_pixels:
                    removed_too_large += 1
                    continue
                img.verify()
            shutil.copy2(file_path, os.path.join(dst_path, file))
            valid_images += 1
        except Exception:
            removed_invalid += 1

print(f"\nImage filtering complete. Valid: {valid_images}, Removed: {total_images - valid_images}")


Image filtering complete. Valid: 11972, Removed: 11


🔹 3. CLIP-Based Filtering (Similarity)

In [ ]:
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

dest_dir = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_filtered"
dest_dir_clip = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_final"
dest_dir_removed = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_removed"

def get_clip_embedding(image_path):
    try:
        img = Image.open(image_path).convert("RGB")
        img_tensor = preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            return model.encode_image(img_tensor).cpu().numpy()[0]
    except Exception as e:
        print(f" Error processing {image_path}: {e}")
        return None

# Track removals
removal_counter = defaultdict(int)
total_removed = 0
total_processed = 0

folder_counter = 0
for folder in tqdm(os.listdir(dest_dir), desc="CLIP cleaning"):
    folder_counter += 1
    fpath = os.path.join(dest_dir, folder)
    f_keep = os.path.join(dest_dir_clip, folder)
    f_rm = os.path.join(dest_dir_removed, folder)
    os.makedirs(f_keep, exist_ok=True)
    os.makedirs(f_rm, exist_ok=True)

    image_paths, embeddings = [], []
    for file in os.listdir(fpath):
        full_path = os.path.join(fpath, file)
        emb = get_clip_embedding(full_path)
        if emb is not None:
            image_paths.append(full_path)
            embeddings.append(emb)

    total_processed += len(image_paths)

    if len(embeddings) < 3:
        for img in image_paths:
            shutil.copy2(img, os.path.join(f_keep, os.path.basename(img)))
        print(f"\n {folder}: Kept all {len(image_paths)} images (too few for clustering)")
        continue

    embeddings = np.vstack(embeddings)

    # Clustering with DBSCAN
    db = DBSCAN(eps=0.3, min_samples=2, metric="cosine").fit(embeddings)
    labels = db.labels_

    if len(set(labels)) == 1 and -1 in labels:
        print(f"\n {folder}: All images marked as noise. Keeping all.")
        for img in image_paths:
            shutil.copy2(img, os.path.join(f_keep, os.path.basename(img)))
        continue

    label_counts = Counter(labels)
    label_counts.pop(-1, None)
    if not label_counts:
        print(f"\n {folder}: No valid clusters found. Keeping all.")
        for img in image_paths:
            shutil.copy2(img, os.path.join(f_keep, os.path.basename(img)))
        continue

    majority_label = label_counts.most_common(1)[0][0]
    kept_files = []
    removed_files = []

    for i, img_path in enumerate(image_paths):
        filename = os.path.basename(img_path)
        if labels[i] == majority_label:
            shutil.copy2(img_path, os.path.join(f_keep, filename))
            kept_files.append(filename)
        else:
            shutil.copy2(img_path, os.path.join(f_rm, filename))
            removed_files.append(filename)

    removal_counter[folder] += len(removed_files)
    total_removed += len(removed_files)

    print(f"\n {folder}:")
    print(f"   Kept: {len(kept_files)}/{len(image_paths)} ({100 * len(kept_files) / len(image_paths):.1f}%)")
    print(f"   Removed: {len(removed_files)}")

    if folder_counter <= 5:
        print(f" Examples:")
        print(f"    Kept    : {kept_files[:5]}")
        print(f"    Removed : {removed_files[:5]}")

# Final Summary
print("\n===  CLIP + Clustering Complete ===")
print(f" Final images saved to: {dest_dir_clip}")
print(f" Removed images saved to: {dest_dir_removed}")

print("\n Removal Summary Per Family:")
for family, removed in removal_counter.items():
    print(f" - {family}: {removed} images removed")

print(f"\n Total Removed: {total_removed} / {total_processed} images")
print(f" Percentage Removed: {100 * total_removed / total_processed:.2f}%")



CLIP cleaning:   0%|          | 1/202 [00:24<1:22:34, 24.65s/it]


 arthropoda_apidae:
   Kept: 150/150 (100.0%)
   Removed: 0
  🔎 Examples:
    Kept    : ['20647903_1065369_eol-full-size-copy.jpg', '20647904_1065369_eol-full-size-copy.jpg', '20647909_1065369_eol-full-size-copy.jpg', '20647915_1065369_eol-full-size-copy.jpg', '21834153_1065290_eol-full-size-copy.jpg']
    Removed : []


CLIP cleaning:   1%|          | 2/202 [00:29<43:46, 13.13s/it]  


 arthropoda_attelabidae:
   Kept: 29/30 (96.7%)
   Removed: 1
  🔎 Examples:
    Kept    : ['14088426_52691998_eol-full-size-copy.jpg', '14088427_52691998_eol-full-size-copy.jpg', '14088429_52691998_eol-full-size-copy.jpg', '14088430_52691998_eol-full-size-copy.jpg', '22864939_52691998_eol-full-size-copy.jpg']
    Removed : ['29600192_52691998_eol-full-size-copy.jpg']


CLIP cleaning:   1%|▏         | 3/202 [00:38<36:42, 11.07s/it]

...


 mollusca_haliotidae:
   Kept: 30/30 (100.0%)
   Removed: 0


CLIP cleaning: 100%|█████████▉| 201/202 [32:52<00:06,  6.94s/it]


 mollusca_unionidae:
   Kept: 60/60 (100.0%)
   Removed: 0


CLIP cleaning: 100%|██████████| 202/202 [32:58<00:00,  9.79s/it]


 mollusca_zonitidae:
   Kept: 29/30 (96.7%)
   Removed: 1

===  CLIP + Clustering Complete ===
 Final images saved to: C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_final
 Removed images saved to: C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_removed

📊 Removal Summary Per Family:
 - arthropoda_apidae: 0 images removed
 - arthropoda_attelabidae: 1 images removed
 - arthropoda_carabidae: 1 images removed
 - arthropoda_cerambycidae: 1 images removed
 - arthropoda_coenagrionidae: 0 images removed
 - arthropoda_formicidae: 38 images removed
 - arthropoda_gomphidae: 0 images removed
 - arthropoda_lucanidae: 0 images removed
 - arthropoda_nymphalidae: 0 images removed
 - arthropoda_palinuridae: 0 images removed
 - arthropoda_papilionidae: 0 images removed
 - arthropoda_pisauridae: 1 images removed
 - arthropoda_platystictidae: 0 images removed
 - arthropoda_pseudophasmatidae: 1 images removed
 - arthropoda_tettigoniidae: 

🔹 4. Dataset Split (Balanced)

In [ ]:
# Paths
source_dir = r"C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_final"
base_output_dir = os.path.join(Path(source_dir).parent, "dataset_sample_stratified")
train_dir = os.path.join(base_output_dir, "train")
val_dir = os.path.join(base_output_dir, "val")
test_dir = os.path.join(base_output_dir, "test")

# Setup output dirs
for d in [train_dir, val_dir, test_dir]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# Collect image paths
image_paths = []
labels = []
valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

for species in os.listdir(source_dir):
    species_dir = os.path.join(source_dir, species)
    if not os.path.isdir(species_dir):
        continue

    species_images = [
        os.path.join(species_dir, img)
        for img in os.listdir(species_dir)
        if Path(img).suffix.lower() in valid_extensions
    ]

    # Sample 60% of images from this species
    random.seed(42)
    random.shuffle(species_images)
    sample_size = int(len(species_images) * 1)
    sampled_images = species_images[:sample_size]

    image_paths.extend(sampled_images)
    labels.extend([species] * len(sampled_images))

# Stratified Split (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(
    image_paths, labels, test_size=0.30, stratify=labels, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

# Copy files & metadata creation
def copy_files(X, y, target_dir, split_name, meta_list):
    for src, label in zip(X, y):
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        dst = os.path.join(label_dir, os.path.basename(src))
        shutil.copy2(src, dst)

        # Extract phylum and family
        if "_" in label:
            phylum, family = label.split("_", 1)
        else:
            phylum, family = "unknown", label

        meta_list.append({
            "file_path": dst,
            "phylum": phylum,
            "family": family,
            "split": split_name
        })

metadata_entries = []
copy_files(X_train, y_train, train_dir, "train", metadata_entries)
copy_files(X_val, y_val, val_dir, "val", metadata_entries)
copy_files(X_test, y_test, test_dir, "test", metadata_entries)

# Save metadata
metadata_df = pd.DataFrame(metadata_entries)
metadata_csv_path = os.path.join(base_output_dir, "metadata.csv")
metadata_df.to_csv(metadata_csv_path, index=False)

# Summary counts
def count_summary(df, split):
    fam_counts = df[df.split == split]["family"].value_counts()
    print(f"\n {split.upper()} SET — Family counts (top 10 shown):")
    print(fam_counts.head(10))
    print(f"Total families in {split}: {df[df.split == split]['family'].nunique()}")

print(" Stratified 60% sampling and dataset generation complete:")
print(f" - Source folder: {source_dir}")
print(f" - Output folder: {base_output_dir}")
print(f" - Metadata CSV: {metadata_csv_path}")
print(f" - Total images: {len(metadata_df)}")
print(f"   → train: {len(X_train)} | val: {len(X_val)} | test: {len(X_test)}")

# Print family class balance
count_summary(metadata_df, "train")
count_summary(metadata_df, "val")
count_summary(metadata_df, "test")

 Stratified 60% sampling and dataset generation complete:
 - Source folder: C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\rare_species_final
 - Output folder: C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\dataset_sample_stratified
 - Metadata CSV: C:\Users\Elcano\Documents\NOVA IMS\2_Semester\Deep Learning\Project\dataset_sample_stratified\metadata.csv
 - Total images: 11472
   → train: 8030 | val: 1721 | test: 1721

🔎 TRAIN SET — Family counts (top 10 shown):
family
dactyloidae        210
cercopithecidae    209
salamandridae      189
plethodontidae     189
carcharhinidae     187
formicidae         177
bovidae            164
acroporidae        146
bucerotidae        143
diomedeidae        119
Name: count, dtype: int64
Total families in train: 202

🔎 VAL SET — Family counts (top 10 shown):
family
cercopithecidae    45
dactyloidae        45
plethodontidae     41
salamandridae      41
carcharhinidae     40
formicidae         38
bovidae          